In [ ]:
#goal: have a table that looks like this
#how to 

'''
|country|first_idx|second_idx|third_idx|

switzerland 1       13         12



////

then incorporate weights



tables DONE:
* salary
* safety
* life expectancy



'''









In [26]:
import pandas as pd
#TODO 


sal = pd.read_csv("salary.csv", delimiter = ';', header = None, index_col = False)
sal.rename(columns = {0: 'salary idx', 1: 'Country', 2: 'Salary', 3: 'Currency', 4: 'Salary[USD]'}, inplace = True)
sal.sort_values(by = 'Salary[USD]', ascending = False, inplace = True)
sal.reset_index(drop=True, inplace=True)  
sal.head()







,salary idx,Country,Salary,Currency,Salary[USD]
0,210.0,Switzerland,11141.0,CHF,11238.0
1,229.0,United States,7772.0,USD,9489.0
2,58.0,Denmark,51990.0,DKK,7640.0
3,107.0,Japan,802090.0,JPY,7502.0
4,125.0,Luxembourg,6663.0,EUR,7313.0


In [20]:
#already sorted

saf = pd.read_csv("scrapers/safety.csv", delimiter = ';', header = None, index_col = False)
saf.rename(columns = {0: 'safety idx', 1: 'Country', 2: 'Score'}, inplace = True)
saf.drop(columns = "Score", axis = 1, inplace = True)
saf.head()



,safety idx,Country
0,1,Iceland
1,2,New Zealand
2,3,Portugal
3,4,Austria
4,5,Denmark


In [21]:
#already sorted

life_exp = pd.read_csv("scrapers/life-exp.csv", delimiter = ';', header = None, index_col = False)
life_exp.rename(columns = {0: 'life exp idx', 1: 'Country', 2: 'Predicted Score'}, inplace = True)
life_exp.drop(columns = "Predicted Score", inplace = True)
life_exp.head()

,life exp idx,Country
0,1,Japan
1,2,Switzerland
2,3,Singapore
3,4,Australia
4,5,Spain


In [47]:
days_off = pd.read_csv("scrapers/days_off.csv", delimiter = ';', header = None, index_col = False)
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)



days_off.head()

,Country,Days Off
0,Iran,53.0
1,Kuwait,48.0
2,Russia,47.0
3,Andorra,45.0
4,Cambodia,45.0


In [56]:
days_off.iloc[[50]]
days_off[days_off['Country']=="United States"].index.item() 


#days_off.index[days_off.loc[days_off['Country'] == 'Sri Lanka']]
#days_off.index['Country' == 'Sri Lanka']
#days_off.loc[['Sri Lanka']]

187